## 6. Dicas de Performance

**✓ Sempre:**
- Use filtros específicos (WHERE)
- Selecione apenas colunas necessárias
- Habilite profiling para análise
- Configure row group size adequado
- Use compression apropriada (SNAPPY para queries, ZSTD para storage)

**✗ Evite:**
- SELECT * em tabelas grandes
- Queries sem filtros em produção
- Row groups muito pequenos (< 1MB) ou grandes (> 1GB)

In [ ]:
import duckdb
import time
from dataclasses import dataclass
from typing import List

@dataclass
class QueryMetrics:
    query: str
    execution_time: float
    rows_processed: int

class PerformanceMonitor:
    """Monitor de performance para queries"""
    
    def __init__(self, con: duckdb.DuckDBPyConnection):
        self.con = con
        self.metrics: List[QueryMetrics] = []
    
    def execute_and_measure(self, query: str) -> QueryMetrics:
        """Executar query e coletar métricas"""
        self.con.execute("SET enable_profiling=true")
        
        start = time.time()
        result = self.con.execute(query).fetchdf()
        execution_time = time.time() - start
        
        metrics = QueryMetrics(
            query=query[:80],
            execution_time=execution_time,
            rows_processed=len(result)
        )
        
        self.metrics.append(metrics)
        return metrics
    
    def print_summary(self):
        """Imprimir resumo"""
        print("\n=== PERFORMANCE SUMMARY ===")
        total_time = sum(m.execution_time for m in self.metrics)
        print(f"Total queries: {len(self.metrics)}")
        print(f"Total time: {total_time:.2f}s")
        print(f"Average time: {total_time/len(self.metrics):.2f}s")
        
        print("\n=== SLOWEST QUERIES ===")
        sorted_metrics = sorted(
            self.metrics,
            key=lambda m: m.execution_time,
            reverse=True
        )[:3]
        
        for i, m in enumerate(sorted_metrics, 1):
            print(f"{i}. {m.execution_time:.2f}s - {m.query}...")

# Exemplo de uso
con = duckdb.connect()
monitor = PerformanceMonitor(con)

queries = [
    "SELECT COUNT(*) FROM sample",
    "SELECT region, COUNT(*) FROM sample GROUP BY region",
    "SELECT * FROM sample WHERE date = '2024-01-15'"
]

for query in queries:
    metrics = monitor.execute_and_measure(query)
    print(f"✓ Query: {metrics.execution_time:.3f}s")

monitor.print_summary()

## 5. Monitor de Performance Completo

Classe para monitorar e analisar performance de queries.

In [ ]:
import duckdb
import time

con = duckdb.connect(':memory:')

# Benchmark: queries repetidas em dados in-memory
start = time.time()
for _ in range(10):
    con.execute("SELECT region, COUNT(*) FROM sample GROUP BY region").fetchall()
time_native = time.time() - start

print(f"DuckDB nativo: {time_native:.3f}s (10 queries)")
print("\n✓ Para queries intensivas, considere carregar dados para DuckDB nativo")

## 4. Comparação: Delta vs DuckDB Nativo

Delta Lake tem overhead adicional - para queries intensivas, carregar para DuckDB nativo pode ser mais rápido.

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

# Criar dados exemplo
df = pd.DataFrame({
    'id': range(10000),
    'value': ['data-' + str(i) for i in range(10000)]
})

# Configurar escrita otimizada
table = pa.Table.from_pandas(df)

# Escrever com SNAPPY (rápido para queries frequentes)
pq.write_table(
    table,
    './optimized_snappy.parquet',
    compression='snappy',
    row_group_size=10000  # Ajustar conforme necessidade
)

# Escrever com ZSTD (melhor compressão)
pq.write_table(
    table,
    './optimized_zstd.parquet',
    compression='zstd',
    compression_level=3,
    row_group_size=10000
)

print("✓ Arquivos otimizados criados")
print("  - SNAPPY: queries rápidas")
print("  - ZSTD: armazenamento eficiente")

## 3. Row Group Size e Compression

Configurar tamanho de row groups e compressão para otimizar performance.

In [ ]:
import duckdb
import time

con = duckdb.connect()

# Teste 1: SELECT *
start = time.time()
df1 = con.execute("SELECT * FROM sample LIMIT 10000").df()
time1 = time.time() - start

# Teste 2: Projeção específica
start = time.time()
df2 = con.execute("SELECT id, region, amount FROM sample LIMIT 10000").df()
time2 = time.time() - start

print(f"SELECT *           : {time1:.3f}s")
print(f"Projeção específica: {time2:.3f}s")
print(f"\n✓ Speedup: {time1/time2:.2f}x")

## 2. Projection Pushdown

Selecionar apenas colunas necessárias reduz I/O drasticamente.

In [ ]:
import duckdb
import time

con = duckdb.connect()

# Criar tabela exemplo
con.execute("""
    CREATE TABLE sample AS
    SELECT 
        i as id,
        'Region-' || (i % 5) as region,
        DATE '2024-01-01' + (i % 365) * INTERVAL '1 day' as date,
        random() * 1000 as amount
    FROM range(100000) t(i)
""")

# Habilitar profiling
con.execute("SET enable_profiling=true")
con.execute("SET profiling_mode='detailed'")

# Query com filter pushdown
start = time.time()
result = con.execute("""
    SELECT COUNT(*), SUM(amount)
    FROM sample
    WHERE date = '2024-01-15' AND region = 'Region-2'
""").fetchone()
elapsed = time.time() - start

print(f"Resultado: {result}")
print(f"Tempo: {elapsed:.3f}s")
print("\n✓ Filtros aplicados antes da leitura completa")

## 1. Filter Pushdown

DuckDB automaticamente empurra filtros para o Delta Lake, reduzindo dados lidos.

In [ ]:
# Instalação de dependências
%pip install duckdb deltalake pyarrow pandas -q

# Capítulo 07: Otimizações e Performance

Técnicas para maximizar performance em queries Delta Lake com DuckDB.

# Capitulo 07 Otimizacoes Performance

Notebook gerado automaticamente a partir do código fonte python.
